In [1]:
import os
import sys
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm
import math
import random
from collections import defaultdict
from typing import List, Tuple
sns.set()
%matplotlib inline

In [2]:
df = pd.read_csv("ta_feng_all_months_merged.csv")

In [3]:
df

,TRANSACTION_DT,CUSTOMER_ID,AGE_GROUP,PIN_CODE,PRODUCT_SUBCLASS,PRODUCT_ID,AMOUNT,ASSET,SALES_PRICE
0,11/1/2000,1104905,45-49,115,110411,4710199010372,2,24,30
1,11/1/2000,418683,45-49,115,120107,4710857472535,1,48,46
2,11/1/2000,1057331,35-39,115,100407,4710043654103,2,142,166
3,11/1/2000,1849332,45-49,Others,120108,4710126092129,1,32,38
4,11/1/2000,1981995,50-54,115,100205,4710176021445,1,14,18
...,...,...,...,...,...,...,...,...,...
817736,2/28/2001,312790,35-39,114,530501,4713317035042,2,80,118
817737,2/28/2001,57486,40-44,115,530209,4710731060124,1,40,55
817738,2/28/2001,733526,>65,Unknown,510539,4716340052307,1,78,115
817739,2/28/2001,173704,45-49,115,520457,4714276145315,1,90,96


In [4]:
df2 = df.copy()

In [5]:
df2 = df2[["CUSTOMER_ID", "PRODUCT_ID", "PRODUCT_SUBCLASS", "TRANSACTION_DT"]]
df2

,CUSTOMER_ID,PRODUCT_ID,PRODUCT_SUBCLASS,TRANSACTION_DT
0,1104905,4710199010372,110411,11/1/2000
1,418683,4710857472535,120107,11/1/2000
2,1057331,4710043654103,100407,11/1/2000
3,1849332,4710126092129,120108,11/1/2000
4,1981995,4710176021445,100205,11/1/2000
...,...,...,...,...
817736,312790,4713317035042,530501,2/28/2001
817737,57486,4710731060124,530209,2/28/2001
817738,733526,4716340052307,510539,2/28/2001
817739,173704,4714276145315,520457,2/28/2001


In [6]:
# process month
df2['s1'] = df2["TRANSACTION_DT"].str.startswith('11/')
df2['s2'] = df2["TRANSACTION_DT"].str.startswith('12/')
df2['s3'] = df2["TRANSACTION_DT"].str.startswith('1/')
df2['s4'] = df2["TRANSACTION_DT"].str.startswith('2/')
df2

,CUSTOMER_ID,PRODUCT_ID,PRODUCT_SUBCLASS,TRANSACTION_DT,s1,s2,s3,s4
0,1104905,4710199010372,110411,11/1/2000,True,False,False,False
1,418683,4710857472535,120107,11/1/2000,True,False,False,False
2,1057331,4710043654103,100407,11/1/2000,True,False,False,False
3,1849332,4710126092129,120108,11/1/2000,True,False,False,False
4,1981995,4710176021445,100205,11/1/2000,True,False,False,False
...,...,...,...,...,...,...,...,...
817736,312790,4713317035042,530501,2/28/2001,False,False,False,True
817737,57486,4710731060124,530209,2/28/2001,False,False,False,True
817738,733526,4716340052307,510539,2/28/2001,False,False,False,True
817739,173704,4714276145315,520457,2/28/2001,False,False,False,True


In [7]:
df2['m'] = df2["TRANSACTION_DT"].str.split('/').apply(lambda x: x[0])

In [8]:
# process by 2week range
df2['half_m'] = df2["TRANSACTION_DT"].str.split('/').apply(lambda x: 0 if int(x[1]) < 16 else 1)  # further session split based on half month

In [9]:
# obtain subclass and id maps
subclass_map = {}
for i, val in enumerate(sorted(list(df2.PRODUCT_SUBCLASS.unique()))):
    subclass_map[val] = i+1
id_map = {}
for j, val2 in enumerate(sorted(list(df2.PRODUCT_ID.unique()))):
    id_map[val2] = j+1

In [10]:
# number of unique ids
max(id_map.values())

23812

In [11]:
df2

,CUSTOMER_ID,PRODUCT_ID,PRODUCT_SUBCLASS,TRANSACTION_DT,s1,s2,s3,s4,m,half_m
0,1104905,4710199010372,110411,11/1/2000,True,False,False,False,11,0
1,418683,4710857472535,120107,11/1/2000,True,False,False,False,11,0
2,1057331,4710043654103,100407,11/1/2000,True,False,False,False,11,0
3,1849332,4710126092129,120108,11/1/2000,True,False,False,False,11,0
4,1981995,4710176021445,100205,11/1/2000,True,False,False,False,11,0
...,...,...,...,...,...,...,...,...,...,...
817736,312790,4713317035042,530501,2/28/2001,False,False,False,True,2,1
817737,57486,4710731060124,530209,2/28/2001,False,False,False,True,2,1
817738,733526,4716340052307,510539,2/28/2001,False,False,False,True,2,1
817739,173704,4714276145315,520457,2/28/2001,False,False,False,True,2,1


In [12]:
# substitute original id with generated ids
df2["PRODUCT_ID"] = df2["PRODUCT_ID"].apply(lambda x: id_map[x])
df2["PRODUCT_SUBCLASS"] = df2["PRODUCT_SUBCLASS"].apply(lambda x: subclass_map[x])

In [13]:
df2

,CUSTOMER_ID,PRODUCT_ID,PRODUCT_SUBCLASS,TRANSACTION_DT,s1,s2,s3,s4,m,half_m
0,1104905,7553,216,11/1/2000,True,False,False,False,11,0
1,418683,10584,276,11/1/2000,True,False,False,False,11,0
2,1057331,5569,62,11/1/2000,True,False,False,False,11,0
3,1849332,6933,277,11/1/2000,True,False,False,False,11,0
4,1981995,7327,18,11/1/2000,True,False,False,False,11,0
...,...,...,...,...,...,...,...,...,...,...
817736,312790,14265,1024,2/28/2001,False,False,False,True,2,1
817737,57486,10324,991,2/28/2001,False,False,False,True,2,1
817738,733526,18028,834,2/28/2001,False,False,False,True,2,1
817739,173704,15779,938,2/28/2001,False,False,False,True,2,1


In [14]:
map_df = df2[["PRODUCT_ID","PRODUCT_SUBCLASS"]]
side_info_map = map_df.set_index("PRODUCT_ID").to_dict()['PRODUCT_SUBCLASS']
len(set(side_info_map.values())) # number of unique side information ids

2011

In [15]:
side_info_array = np.zeros(len(side_info_map))
for i in range(len(side_info_array)):
    side_info_array[i] = side_info_map[i+1]
side_info_array # to be used 

array([ 706.,  706.,  706., ..., 1669., 1669., 1669.])

In [16]:
# save the mapping
import pickle
with open('tafeng_item_to_side_index.pkl','wb') as f:
     pickle.dump(side_info_array, f)

In [17]:
df3 = df2.groupby(['CUSTOMER_ID','m','half_m']).agg({'PRODUCT_SUBCLASS': lambda x: x.tolist(), 'PRODUCT_ID': lambda x: x.tolist()}).reset_index()
df3

,CUSTOMER_ID,m,half_m,PRODUCT_SUBCLASS,PRODUCT_ID
0,1069,1,1,"[201, 42, 201]","[8243, 5145, 13561]"
1,1069,11,0,"[18, 45]","[23573, 7318]"
2,1069,2,0,"[180, 134, 45, 1, 275, 143]","[12886, 6284, 7318, 2767, 4836, 21049]"
3,1113,1,0,"[240, 161, 240, 43, 206, 238, 135]","[7881, 4802, 7882, 2914, 20524, 23514, 6288]"
4,1113,11,0,"[206, 132]","[20763, 11316]"
...,...,...,...,...,...
83289,20002000,11,0,"[275, 275, 275, 331, 425]","[19407, 19400, 19406, 16787, 6824]"
83290,20002000,11,1,"[400, 182, 18, 45, 1, 967, 182, 172, 131, 226,...","[21899, 10468, 6537, 9083, 8833, 6434, 10466, ..."
83291,20002000,12,0,"[107, 1535, 224, 951, 1535, 136, 313, 1814, 10...","[5507, 18598, 8245, 14288, 18599, 9444, 6692, ..."
83292,20002000,12,1,"[690, 1058, 1118, 1914, 967, 316, 307, 1635, 1...","[15188, 364, 22624, 17047, 11322, 10950, 4438,..."


In [18]:
df3['session_len'] = df3['PRODUCT_SUBCLASS'].str.len()
df3

,CUSTOMER_ID,m,half_m,PRODUCT_SUBCLASS,PRODUCT_ID,session_len
0,1069,1,1,"[201, 42, 201]","[8243, 5145, 13561]",3
1,1069,11,0,"[18, 45]","[23573, 7318]",2
2,1069,2,0,"[180, 134, 45, 1, 275, 143]","[12886, 6284, 7318, 2767, 4836, 21049]",6
3,1113,1,0,"[240, 161, 240, 43, 206, 238, 135]","[7881, 4802, 7882, 2914, 20524, 23514, 6288]",7
4,1113,11,0,"[206, 132]","[20763, 11316]",2
...,...,...,...,...,...,...
83289,20002000,11,0,"[275, 275, 275, 331, 425]","[19407, 19400, 19406, 16787, 6824]",5
83290,20002000,11,1,"[400, 182, 18, 45, 1, 967, 182, 172, 131, 226,...","[21899, 10468, 6537, 9083, 8833, 6434, 10466, ...",192
83291,20002000,12,0,"[107, 1535, 224, 951, 1535, 136, 313, 1814, 10...","[5507, 18598, 8245, 14288, 18599, 9444, 6692, ...",204
83292,20002000,12,1,"[690, 1058, 1118, 1914, 967, 316, 307, 1635, 1...","[15188, 364, 22624, 17047, 11322, 10950, 4438,...",41


In [19]:
df3['session_len'].max()  

390

In [20]:
product_subclass_sessions = list(df3.PRODUCT_SUBCLASS)
product_id_sessions = list(df3.PRODUCT_ID)

In [21]:
processed_subclass_sessions = []
for session in tqdm(product_subclass_sessions):
    if len(session) == 1:
        continue
    elif len(session) < 40:   # tentative session length 
        processed_subclass_sessions.append(session)
    else:
        for i in range(math.ceil(len(session) / 40)):
            processed_subclass_sessions.append(session[i*40:(i+1)*40])

100%|██████████| 83294/83294 [00:00<00:00, 1332510.84it/s]


In [22]:
processed_id_sessions = []
for session in tqdm(product_id_sessions):
    if len(session) == 1:
        continue
    elif len(session) < 40:   # tentative session length 
        processed_id_sessions.append(session)
    else:
        for i in range(math.ceil(len(session) / 40)):
            processed_id_sessions.append(session[i*40:(i+1)*40])

100%|██████████| 83294/83294 [00:00<00:00, 1440774.15it/s]


In [23]:
len(processed_subclass_sessions)

76762

In [24]:
# finished session generation
# starting train/val/test split
tmp = list(zip(processed_id_sessions, processed_subclass_sessions))
random.shuffle(tmp)
processed_id_sessions, processed_subclass_sessions = zip(*tmp)

In [25]:
# subclass related datasets are generated but not used, can be ignored

train_idx = math.ceil(len(processed_subclass_sessions) * 0.7)
val_idx = math.ceil(len(processed_subclass_sessions) * 0.9)
print(train_idx, val_idx)
subclass_train_set = processed_subclass_sessions[:train_idx]
subclass_val_set = processed_subclass_sessions[train_idx:val_idx]
subclass_test_set = processed_subclass_sessions[val_idx:]
###
id_train_set = processed_id_sessions[:train_idx]
id_val_set = processed_id_sessions[train_idx:val_idx]
id_test_set = processed_id_sessions[val_idx:]

53734 69086


In [26]:
# generating train/val/test instances
file_set_map = {"preprocessed_subclass_train.txt": subclass_train_set, 
                "preprocessed_subclass_val.txt": subclass_val_set,
               "preprocessed_subclass_test.txt": subclass_test_set,
               "preprocessed_id_train.txt": id_train_set, 
                "preprocessed_id_val.txt": id_val_set,
               "preprocessed_id_test.txt": id_test_set}
for k,v in file_set_map.items():
    count = 0
    with open(k, "w") as f:
        for session in v:
    #         print(session)
            for i in range(1,len(session)):
                line = session[:i] + [0] * (40 - i)
                to_write = str(line) + '|' + str([session[i]])
                f.write(to_write)
                f.write('\n')
                count += 1
    print(k, count)

preprocessed_subclass_train.txt 512747
preprocessed_subclass_val.txt 146575
preprocessed_subclass_test.txt 73727
preprocessed_id_train.txt 512747
preprocessed_id_val.txt 146575
preprocessed_id_test.txt 73727


In [28]:
# write side info files
result = []
with open('preprocessed_id_val.txt','r') as f:
    for line in f.readlines():
        x, y = line.rstrip('\n').split('|')
        x_list = x[1:-1].split(', ')
        y = y[1:-1]
        to_append_x = [0] * len(x_list)
        for idx in range(len(x_list)):
            if x_list[idx] == '0':
                break
            else:
                to_append_x[idx] = int(side_info_array[int(x_list[idx])-1])
        to_append = str(to_append_x) + '|' + str([int(side_info_array[int(y)-1])]) + '\n'
        result.append(to_append) 
    
with open("tafeng_side_info_val.txt", "w") as f:
    f.writelines(result)